In [20]:
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fastapi 라이브러리
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi import File, UploadFile
from fastapi.responses import JSONResponse, HTMLResponse


# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

# 모델 Load 용 라이브러리
import PCB_Product_Model
import PCB_Defect_Model

# 이미지 웹 서버로 보내주기 위함
from PIL import Image
import io
import base64

# 데이터 분석용 라이브러리
import pandas as pd
import numpy as np
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# ngrok을 활용하여 분석서버를 Global하게 띄우기 위한 라이브러리
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [21]:
# CORS issue 때문에 미들웨어 추가

from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

In [22]:
# 제품 타입 분류

In [23]:
model_path = 'pcb_prod_model_0708.h5'
class_path = 'pcb_class_dict_0708.json'

pcb_prod_model = PCB_Product_Model.Model_load(model_path, class_path)


In [24]:
# 양불 판정

In [25]:
model_pt_path = 'best.pt'
pcb_defect_model = PCB_Defect_Model.Model_load(model_pt_path)



In [26]:
def preprocess_image(image: Image.Image) -> np.ndarray:
    # Convert the image to a NumPy array
    img_array = np.array(image)
    return img_array

def ndarray_to_base64(arr: np.ndarray) -> str:
    _, buffer = cv2.imencode('.jpg', arr)
    img_str = base64.b64encode(buffer).decode('utf-8')
    return img_str

In [28]:
@app.get("/", response_class=HTMLResponse)
async def main():
    content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Image Upload</title>
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    </head>
    <body>
        <h1>Upload an Image</h1>
        <form id="uploadForm">
            <input type="file" id="fileInput" name="file"><br><br>
            <button type="button" id="uploadButton">Upload</button>
        </form>
        <div id="result"></div>

        <script>
            $(document).ready(function() {
                $('#uploadButton').click(function() {
                    var formData = new FormData();
                    var fileInput = $('#fileInput')[0];
                    if (fileInput.files.length === 0) {
                        alert('Please select a file');
                        return;
                    }

                    formData.append('file', fileInput.files[0]);

                    $.ajax({
                        url: '/predict',
                        type: 'POST',
                        data: formData,
                        contentType: false,
                        processData: false,
                        success: function(response) {
                            console.log(response)
                            //$('#result').html('<p>' + response.message + '</p><p>Filename: ' + response.filename + '</p>');
                        },
                        error: function(response) {
                            //$('#result').html('<p>' + response.responseJSON.error + '</p>');
                        }
                    });
                });
            });
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=content)


@app.post("/predict")
async def upload_image(file: UploadFile = File(...)):

    contents = await file.read()

    # 이미지 읽기
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    prod_type = pcb_prod_model.predict_image(False, img)

    prod_defect_dict = pcb_defect_model.predict_defect(img)

    error_type_list = prod_defect_dict['defect_name_list']
    defect_box_list = prod_defect_dict['image']
    
    # img_str = ndarray_to_base64(defect_image)
    
    result = {
        "product_type": prod_type,
        "error_type_list": error_type_list,
        "image": defect_box_list
    }

    return result


In [29]:
# colab 서버 주소는 실행 될 때마다 바뀌기 때문에 ngrok이라는 것을 활용하여서 고정적인 url로 들어올 수 있는 서버를 만들 수 있다. 아래의 코드를 활용하면 된다.

import nest_asyncio
from pyngrok import ngrok
import uvicorn

AUTH_TOKEN = "2hRkhpGbfhVbsBcpYewFZne3I4z_24P9h5EchPjzVTMJe6zEL"
ngrok.set_auth_token(AUTH_TOKEN)
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)



공용 URL https://c30d-118-34-210-82.ngrok-free.app


INFO:     Started server process [31112]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     118.34.210.82:0 - "GET / HTTP/1.1" 200 OK
INFO:     118.34.210.82:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
1/1 [==============================] - 0s 71ms/step

0: 352x640 2 shorts, 109.8ms
Speed: 3.0ms preprocess, 109.8ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict5

0: 352x640 2 shorts, 114.8ms
Speed: 2.2ms preprocess, 114.8ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 2 shorts, 102.8ms
Speed: 2.4ms preprocess, 102.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)
INFO:     118.34.210.82:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     211.231.103.107:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     211.231.103.94:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [31112]
t=2024-07-15T16:35:37+0900 lvl=warn msg="failed to open private leg" id=b7cfc9ad6eb0 privaddr=localhost:8000 err="dial tcp [::1]:8000: connectex: No connection could be made because the target machine actively refused it."
t=2024-07-15T16:41:28+0900 lvl=warn msg="failed to open private leg" id=6e128bdb421a privaddr=localhost:8000 err="dial tcp [::1]:8000: connectex: No connection could be made because the target machine actively refused it."


In [10]:
imgPath = "../PCB_DATASET/images/short/01_short_02.jpg"
img = cv2.imread(imgPath)


In [11]:
prod_defect_dict = pcb_defect_model.predict_defect(img)


0: 352x640 3 shorts, 142.1ms
Speed: 3.1ms preprocess, 142.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict4

0: 352x640 3 shorts, 99.4ms
Speed: 1.0ms preprocess, 99.4ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 shorts, 94.1ms
Speed: 2.0ms preprocess, 94.1ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)


In [16]:
error_type_list = prod_defect_dict['defect_name_list']
defect_image = prod_defect_dict['image']
    

In [17]:
type(error_type_list)

list

In [18]:
type(defect_image)

numpy.ndarray